<a href="https://colab.research.google.com/github/icdath95/DATA-analyst/blob/sub/%ED%8C%80%EB%B3%84_%EC%8B%A4%EC%A0%81_%EB%B9%84%EA%B5%90_%EB%B6%84%EC%84%9D%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

file = '/content/KBO_2019_player_gamestats.csv'

raw = pd.read_csv(file, encoding = 'cp949')
raw.head()

,팀,이름,생일,일자,상대,결과,타순,P,선발,타수,득점,안타,2타,3타,홈런,루타,타점,도루,도실,볼넷,사구,고4,삼진,병살,희타,희비,타율,출루,장타,OPS,투구,avLI,RE24,WPA
0,두산,페르난데스,1988-04-27,03-23,한화,W 5:4,6,DH,1,4,0,2,1,0,0,3,3,0,0,0,0,0,0,1,0,0,0.500,0.500,0.750,1.250,19,1.98,1.65,0.429
1,두산,페르난데스,1988-04-27,03-24,한화,L 1:11,6,DH,1,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.333,0.333,0.500,0.833,4,0.77,-0.36,-0.038
2,두산,페르난데스,1988-04-27,03-26,키움,W 7:2,2,DH,1,2,2,0,0,0,0,0,1,0,0,2,0,0,0,0,0,0,0.250,0.400,0.375,0.775,16,1.56,0.98,0.146
3,두산,페르난데스,1988-04-27,03-27,키움,W 3:2,2,DH,1,4,1,3,0,0,0,3,1,0,0,0,0,0,0,0,0,0,0.417,0.500,0.500,1.000,11,1.53,1.29,0.189
4,두산,페르난데스,1988-04-27,03-28,키움,L 4:5,2,DH,1,3,1,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0.333,0.474,0.400,0.874,23,2.04,-0.30,-0.166


In [ ]:
# 상대팀별 기록 정리하기
# 컬럼에 어떤 데이터 있나 살펴보기 .유니크

raw['상대'].unique()

array(['한화', '키움', '@삼성', 'KT', 'NC', '@롯데', '@LG', 'SK', '@KIA', '@키움',
       '롯데', '@한화', 'LG', 'KIA', '@NC', '삼성', '@SK', '@KT', '두산', '@두산'],
      dtype=object)

In [ ]:
# 여기서 @ 붙은거는 원정경기라는 뜻
# '상대' 컬럼에서 홈/원정 여부, 상대팀을 분리하여 각각 '홈어웨이' '상대팀' 컬럼 지정
# ? 왜 내가 작성한건 안되고 복사한건 되는건지... 다시 한번 ㄱ 여긴 복습
opp_list = [ ]
home_away_list = [ ]

for opp in raw['상대']:
    if "@" in opp:
        home_away = '원정'
        opp = opp.replace('@', '')
    else:
        home_away = '홈'
    home_away_list.append(home_away)
    opp_list.append(opp)

raw['홈어웨이'] = home_away_list
raw['상대팀'] = opp_list

raw.head()

,팀,이름,생일,일자,상대,결과,타순,P,선발,타수,득점,안타,2타,3타,홈런,루타,타점,도루,도실,볼넷,사구,고4,삼진,병살,희타,희비,타율,출루,장타,OPS,투구,avLI,RE24,WPA,홈어웨이,상대팀
0,두산,페르난데스,1988-04-27,03-23,한화,W 5:4,6,DH,1,4,0,2,1,0,0,3,3,0,0,0,0,0,0,1,0,0,0.500,0.500,0.750,1.250,19,1.98,1.65,0.429,홈,한화
1,두산,페르난데스,1988-04-27,03-24,한화,L 1:11,6,DH,1,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.333,0.333,0.500,0.833,4,0.77,-0.36,-0.038,홈,한화
2,두산,페르난데스,1988-04-27,03-26,키움,W 7:2,2,DH,1,2,2,0,0,0,0,0,1,0,0,2,0,0,0,0,0,0,0.250,0.400,0.375,0.775,16,1.56,0.98,0.146,홈,키움
3,두산,페르난데스,1988-04-27,03-27,키움,W 3:2,2,DH,1,4,1,3,0,0,0,3,1,0,0,0,0,0,0,0,0,0,0.417,0.500,0.500,1.000,11,1.53,1.29,0.189,홈,키움
4,두산,페르난데스,1988-04-27,03-28,키움,L 4:5,2,DH,1,3,1,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0.333,0.474,0.400,0.874,23,2.04,-0.30,-0.166,홈,키움


In [ ]:
# 상대 팀별 실적을 정리하기 위해 피벗 테이블을 만듬

data = raw.pivot_table( index = ['팀', '이름', '생일', '상대팀'], 
                values = ['타수','안타','홈런', '루타', '타점','볼넷', '사구', '희비'],
                aggfunc = 'sum')

data.head()

루타  볼넷  사구  안타  타수  타점  홈런  희비
팀   이름  생일         상대팀                                
KIA 고영창 1989-02-24 LG    0   0   0   0   0   0   0   0
    김선빈 1989-12-18 KT    5   1   0   4  33   4   0   0
                   LG   15   5   0  12  47   6   0   0
                   NC   14   8   0  14  42   2   0   0
                   SK   10   6   0  10  36   4   0   1

In [ ]:
# 상대팀별 타수가 0보다 큰 경우만 선택

cond = data['타수'] > 0

data = data[cond]

In [ ]:
data = data.reset_index()

data.head()

,팀,이름,생일,상대팀,루타,볼넷,사구,안타,타수,타점,홈런,희비
0,KIA,김선빈,1989-12-18,KT,5,1,0,4,33,4,0,0
1,KIA,김선빈,1989-12-18,LG,15,5,0,12,47,6,0,0
2,KIA,김선빈,1989-12-18,NC,14,8,0,14,42,2,0,0
3,KIA,김선빈,1989-12-18,SK,10,6,0,10,36,4,0,1
4,KIA,김선빈,1989-12-18,두산,20,3,0,15,54,5,0,0


In [ ]:
# 타자 주요 실적을 계산하는 함수 / 이렇게 주로 쓰는 함수, 계속쓰는 함수는 저장해두고 복붙하기
def cal_hit(df):
    '''
    - 타율 : 공을 쳐서 나가는 비율 --> 안타 / 타수
    - 출루율: 진루해서 나가는 비율 -->  (안타+볼넷+몸에맞는볼)/(타수+볼넷+몸에맞는볼+희생플라이)
    - 장타율 : 타율에 진루한 베이스 가중치 추가 -->   루타 / 타수
    '''
    
    df['타율'] = df['안타'] / df['타수']
    df['출루율'] = (df['안타'] + df['볼넷'] + df['사구']) / (df['타수'] + df['사구'] + df['희비'])
    df['장타율'] = df['루타'] / df['타수']
    df['OPS'] = df['출루율'] + df['장타율']
    return df

In [ ]:
# 타자 / 상대팀별 실적을 계싼

player_stat_opp = cal_hit(data)
player_stat_opp.head()

,팀,이름,생일,상대팀,루타,볼넷,사구,안타,타수,타점,홈런,희비,타율,출루율,장타율,OPS
0,KIA,김선빈,1989-12-18,KT,5,1,0,4,33,4,0,0,0.121212,0.151515,0.151515,0.303030
1,KIA,김선빈,1989-12-18,LG,15,5,0,12,47,6,0,0,0.255319,0.361702,0.319149,0.680851
2,KIA,김선빈,1989-12-18,NC,14,8,0,14,42,2,0,0,0.333333,0.523810,0.333333,0.857143
3,KIA,김선빈,1989-12-18,SK,10,6,0,10,36,4,0,1,0.277778,0.432432,0.277778,0.710210
4,KIA,김선빈,1989-12-18,두산,20,3,0,15,54,5,0,0,0.277778,0.333333,0.370370,0.703704


In [ ]:
# 특정팀을 상대로 강한 타자 보기
# 먼저 두산을 상대로 타수가 10보다 많은 선수중 상위 10명뽑기
team = '두산'
cond = (player_stat_opp['상대팀'] == team) & (player_stat_opp['타수'] > 10)
#cond = player_stat_opp['상대팀'] == team & (player_stat_opp['타수'] > 10)

player_stat_opp[cond]

,팀,이름,생일,상대팀,루타,볼넷,사구,안타,타수,타점,홈런,희비,타율,출루율,장타율,OPS
4,KIA,김선빈,1989-12-18,두산,20,3,0,15,54,5,0,0,0.277778,0.333333,0.370370,0.703704
13,KIA,김주찬,1981-03-25,두산,12,0,1,9,37,1,0,0,0.243243,0.263158,0.324324,0.587482
22,KIA,나지완,1985-05-19,두산,11,7,0,5,17,4,1,1,0.294118,0.666667,0.647059,1.313725
31,KIA,류승현,1997-07-01,두산,6,0,1,4,16,1,0,0,0.250000,0.294118,0.375000,0.669118
49,KIA,박찬호,1995-06-05,두산,25,5,1,18,54,4,0,1,0.333333,0.428571,0.462963,0.891534
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1621,한화,정은원,2000-01-17,두산,22,4,0,17,65,10,0,3,0.261538,0.308824,0.338462,0.647285
1630,한화,지성준,1994-04-10,두산,4,2,0,4,18,1,0,0,0.222222,0.333333,0.222222,0.555556
1644,한화,최재훈,1989-08-27,두산,11,3,3,11,36,4,0,1,0.305556,0.425000,0.305556,0.730556
1653,한화,최진행,1985-08-17,두산,0,1,0,0,15,0,0,0,0.000000,0.066667,0.000000,0.066667


In [ ]:
# 출루율 기준 상위 10명 뽑기
player_stat_opp[cond].sort_values(by = "출루율", ascending = False).head(10)

,팀,이름,생일,상대팀,루타,볼넷,사구,안타,타수,타점,홈런,희비,타율,출루율,장타율,OPS
22,KIA,나지완,1985-05-19,두산,11,7,0,5,17,4,1,1,0.294118,0.666667,0.647059,1.313725
190,KT,강백호,1999-07-29,두산,18,10,0,14,42,8,1,0,0.333333,0.571429,0.428571,1.000000
538,NC,나성범,1989-10-03,두산,9,1,1,5,12,2,1,0,0.416667,0.538462,0.750000,1.288462
1664,한화,호잉,1989-05-18,두산,30,11,0,18,54,5,2,0,0.333333,0.537037,0.555556,1.092593
564,NC,박민우,1993-02-06,두산,25,5,1,20,49,7,0,0,0.408163,0.520000,0.510204,1.030204
145,KIA,터커,1990-07-06,두산,16,7,0,9,31,2,1,0,0.290323,0.516129,0.516129,1.032258
573,NC,박석민,1985-06-22,두산,20,7,0,12,37,13,2,0,0.324324,0.513514,0.540541,1.054054
1591,한화,이성열,1984-07-13,두산,30,8,1,20,54,15,1,2,0.370370,0.508772,0.555556,1.064327
1423,키움,샌즈,1987-09-28,두산,39,13,1,23,73,18,2,0,0.315068,0.500000,0.534247,1.034247
239,KT,박경수,1984-03-31,두산,14,6,1,12,37,9,0,0,0.324324,0.500000,0.378378,0.878378


In [ ]:
# 롯데를 상대로 타수가 20보다 큰 타자 중 출루율 10명 뽑기

team = '롯데'
cond = (player_stat_opp['상대팀'] == team) & (player_stat_opp['타수'] > 20)
player_stat_opp[cond].sort_values(by = "출루율", ascending = False).head(10)

,팀,이름,생일,상대팀,루타,볼넷,사구,안타,타수,타점,홈런,희비,타율,출루율,장타율,OPS
330,KT,황재균,1987-07-28,롯데,35,11,0,22,47,12,3,1,0.468085,0.687500,0.744681,1.432181
610,NC,양의지,1987-06-05,롯데,29,7,2,20,42,8,2,0,0.476190,0.659091,0.690476,1.349567
1195,삼성,김상수,1990-03-23,롯데,23,13,2,17,49,5,1,0,0.346939,0.627451,0.469388,1.096839
1433,키움,서건창,1989-08-22,롯데,26,3,0,21,39,7,0,0,0.538462,0.615385,0.666667,1.282051
1225,삼성,러프,1986-07-28,롯데,28,16,1,13,51,15,3,1,0.254902,0.566038,0.549020,1.115057
287,KT,유한준,1981-07-01,롯데,30,11,0,21,56,10,2,1,0.375000,0.561404,0.535714,1.097118
410,LG,이천웅,1988-10-20,롯데,33,10,0,24,61,5,1,0,0.393443,0.557377,0.540984,1.098361
5,KIA,김선빈,1989-12-18,롯데,10,6,0,9,25,3,0,2,0.360000,0.555556,0.400000,0.955556
574,NC,박석민,1985-06-22,롯데,39,12,1,18,54,19,5,1,0.333333,0.553571,0.722222,1.275794
1461,키움,이지영,1986-02-27,롯데,16,2,0,15,31,6,0,0,0.483871,0.548387,0.516129,1.064516


In [ ]:
# kbo 전체 팀을 대상으로 팀별 출루율 상위 5인 타자 살펴보기

hitter_df = pd.DataFrame()

for team in player_stat_opp['상대팀'].unique():

  cond = (player_stat_opp['상대팀'] == team) & (player_stat_opp["타수"] > 20)
  df = player_stat_opp[cond].sort_values(by = '출루율', ascending = False).head()

  hitter_df = hitter_df.append(df)



In [ ]:
hitter_df.head()

,팀,이름,생일,상대팀,루타,볼넷,사구,안타,타수,타점,홈런,희비,타율,출루율,장타율,OPS
1204,삼성,김헌곤,1988-11-09,KT,13,11,1,12,38,3,0,0,0.315789,0.615385,0.342105,0.957490
1401,키움,박병호,1986-07-10,KT,26,14,1,10,38,10,5,2,0.263158,0.609756,0.684211,1.293967
415,LG,이형종,1989-06-07,KT,19,6,1,15,39,3,1,0,0.384615,0.550000,0.487179,1.037179
132,KIA,최형우,1983-12-16,KT,23,10,1,16,48,7,1,1,0.333333,0.540000,0.479167,1.019167
816,SK,한동민,1989-08-09,KT,25,8,1,19,51,10,2,0,0.372549,0.538462,0.490196,1.028658


In [ ]:
# 특정팀 상대 출루율 탑5 이내 들어있는 타자리스트
# unique()명령을 이용해 복수의 팀을 대상으로 탑5에 들어있을경우 한번만 나타나게(중복제거)

hitter_df['이름'].unique()

array(['김헌곤', '박병호', '이형종', '최형우', '한동민', '정근우', '김하성', '터커', '박건우',
       '김재호', '민병헌', '박승욱', '최재훈', '강백호', '오재일', '장영석', '윌슨', '양의지',
       '이원재', '호잉', '박민우', '박석민', '황재균', '김상수', '서건창', '러프', '김혜성', '최정',
       '안치홍', '노수광', '샌즈', '최주환', '오지환'], dtype=object)

In [ ]:
'김현곤' in hitter_df['이름'].unique()

False

In [ ]:
# 특정팀 상대 출루율 탑5 이내 상위 타자들을 대상으로 팀별 출루율 피벗테이블 만들기

cond = []
for name in player_stat_opp['이름']:

  if name in hitter_df['이름'].unique():
    cond.append(True)
  else:
    cond.append(False)

player_stat_opp[cond]

,팀,이름,생일,상대팀,루타,볼넷,사구,안타,타수,타점,홈런,희비,타율,출루율,장타율,OPS
70,KIA,안치홍,1990-07-02,KT,15,1,0,14,49,6,0,0,0.285714,0.306122,0.306122,0.612245
71,KIA,안치홍,1990-07-02,LG,7,5,1,6,22,4,0,0,0.272727,0.521739,0.318182,0.839921
72,KIA,안치홍,1990-07-02,NC,20,3,0,12,39,7,2,0,0.307692,0.384615,0.512821,0.897436
73,KIA,안치홍,1990-07-02,SK,17,3,0,13,47,4,1,1,0.276596,0.333333,0.361702,0.695035
74,KIA,안치홍,1990-07-02,두산,15,2,0,12,40,2,0,0,0.300000,0.350000,0.375000,0.725000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1663,한화,호잉,1989-05-18,SK,20,2,1,15,55,7,1,0,0.272727,0.321429,0.363636,0.685065
1664,한화,호잉,1989-05-18,두산,30,11,0,18,54,5,2,0,0.333333,0.537037,0.555556,1.092593
1665,한화,호잉,1989-05-18,롯데,18,2,1,11,43,6,1,0,0.255814,0.318182,0.418605,0.736786
1666,한화,호잉,1989-05-18,삼성,20,3,1,12,51,8,2,1,0.235294,0.301887,0.392157,0.694044


In [ ]:
# 위에꺼 다른방식으로 확인하기
# isin = 판다스 제공 함수임
# 위에꺼 처럼 개별로 꺼내서 하는법을 일단 추천 이후 판다스 좀더 공부하면 코드를 짧게 이용가능

cond = player_stat_opp['이름'].isin(hitter_df['이름'].unique())
top_df = player_stat_opp[cond]

top_pivot = top_df.pivot_table(index = ['팀', '이름'], values = '출루율',columns = '상대팀')
top_pivot

상대팀           KIA        KT        LG  ...        삼성        키움        한화
팀   이름                                 ...                              
KIA 안치홍       NaN  0.306122  0.521739  ...  0.500000  0.567568  0.420000
    최형우       NaN  0.540000  0.478261  ...  0.615385  0.471698  0.472727
    터커        NaN  0.400000  0.594595  ...  0.317073  0.447368  0.388889
KT  강백호  0.470588       NaN  0.338983  ...  0.448980  0.487805  0.617647
    박승욱  0.210526       NaN  0.222222  ...  0.240000  0.230769  0.285714
    황재균  0.333333       NaN  0.454545  ...  0.448276  0.288462  0.441860
LG  오지환  0.576923  0.446429       NaN  ...  0.423077  0.295082  0.360000
    이형종  0.479167  0.550000       NaN  ...  0.476190  0.444444  0.333333
NC  박민우  0.529412  0.365385  0.397059  ...  0.392857  0.408163  0.577778
    박석민  0.342105  0.500000  0.416667  ...  0.274194  0.594595  0.571429
    양의지  0.720930  0.354167  0.396552  ...  0.340426  0.435897  0.575000
    이원재  0.407407  0.357143  0.250000  ...  0.360000  0.250000  0.375000
SK  노수광  0.326923  0.370370  0.428571  ...  0.352941  0.307692  0.594595
    최정   0.543860  0.457627  0.432836  ...  0.551724  0.338028  0.516667
    한동민  0.333333  0.538462  0.423077  ...  0.314286  0.406250  0.454545
두산  김재호  0.444444  0.486486  0.568182  ...  0.510638  0.516129  0.425532
    박건우  0.297872  0.531915  0.574468  ...  0.509804  0.312500  0.400000
    오재일  0.408163  0.384615  0.254545  ...  0.372881  0.383333  0.534483
    최주환  0.594595  0.218750  0.388889  ...  0.454545  0.326087  0.300000
롯데  민병헌  0.677419  0.459459  0.404255  ...  0.420000  0.204545  0.437500
    윌슨   0.250000  0.269231  0.571429  ...  0.560000  0.451613  0.566667
삼성  김상수  0.535714  0.274510  0.312500  ...       NaN  0.269231  0.390625
    김헌곤  0.259259  0.615385  0.380952  ...       NaN  0.388889  0.625000
    러프   0.388889  0.426230  0.433962  ...       NaN  0.428571  0.573770
키움  김하성  0.402985  0.320755  0.647887  ...  0.353846       NaN  0.379310
    김혜성  0.333333  0.357143  0.339286  ...  0.644444       NaN  0.166667
    박병호  0.620690  0.609756  0.467532  ...  0.560000       NaN  0.446809
    샌즈   0.464286  0.426230  0.361446  ...  0.540984       NaN  0.584906
    서건창  0.450000  0.333333  0.387097  ...  0.395833       NaN  0.454545
    장영석  0.339623  0.333333  0.282051  ...  0.333333       NaN  0.444444
한화  정근우  0.214286  0.309524  0.703704  ...  0.444444  0.300000       NaN
    최재훈  0.531915  0.489362  0.478261  ...  0.306122  0.441860       NaN
    호잉   0.375000  0.380952  0.274510  ...  0.301887  0.438596       NaN

[33 rows x 10 columns]

In [37]:
# 코랩 한글 폰트 설정하는 코드
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 40 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 1s (10.3 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 160837 files and di

In [41]:
# 한ㅓ에 살보 위 히트맵 표
import seaborn as sns

# 사이즈
fig, ax = plt.subplot( figsize = (10,10))
sns.heatmap( top_pivot)

NameError: ignored